<a href="https://colab.research.google.com/github/IntelAntique/Cotiviti/blob/main/Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Dataset
import numpy as np
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
df = pd.read_csv('Diabetes-Data.csv', index_col = False)
df = df.drop(columns="Id")
n = len(df.columns)
dataset = df.values

train, test = torch.utils.data.random_split(dataset, [0.8, 0.2])

In [ ]:
training_data = torch.tensor(train)
test_data = torch.tensor(test)

print(training_data[:, n - 1: n])
print(training_data[:, 0: n - 1])

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]], dtype=torch.float64)
tensor([[2.0000e+00, 1.2500e+02, 6.0000e+01,  ..., 3.3800e+01, 8.8000e-02,
         3.1000e+01],
        [6.0000e+00, 8.5000e+01, 7.8000e+01,  ..., 3.1200e+01, 3.8200e-01,
         4.2000e+01],
        [5.0000e+00, 8.8000e+01, 7.8000e+01,  ..., 2.7600e+01, 2.5800e-01,
         3.7000e+01],
        ...,
        [1.0000e+00, 1.2400e+02, 7.4000e+01,  ..., 2.7800e+01, 1.0000e-01,
         3.0000e+01],
        [2.0000e+00, 5.6000e+01, 5.6000e+01,  ..., 2.4200e+01, 3.3200e-01,
         2.2000e+01],
        [3.0000e+00, 1.7300e+02, 8.2000e+01,  ..., 3.8400e+01, 2.1370e+00,
         2.5000e+01]], dtype=torch.float64)


<ipython-input-12-b514dee7bf96>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  training_data = torch.tensor(train)


In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for data in test_dataloader:
    X, y = data[:, : n - 1], data[:, n - 1]
    print(f"Shape of X [N, C]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C]: torch.Size([64, 8])
Shape of y: torch.Size([64]) torch.float64


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [ ]:
input = 8
output = 2

class BinaryClassification(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.sequential = nn.Sequential(
            nn.Linear(input, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output)
        )

    def forward(self, x):
        return self.sequential(x)

model = BinaryClassification(input, output).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, data in enumerate(dataloader):
        X, y = data[:, : n - 1].type(torch.float).to(device), data[:, n - 1].type(torch.long).to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for data in dataloader:
            X, y = data[:, : n - 1].type(torch.float).to(device), data[:, n - 1].type(torch.long).to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 1000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Streaming output truncated to the last 5000 lines.
        [ 5.8794, -0.6310],
        [ 3.5515, -0.4465],
        [ 1.1585,  3.6946],
        [ 1.7986,  1.2019],
        [ 2.2664,  3.5342],
        [-1.9821,  4.8313],
        [ 2.6160,  2.2471],
        [ 1.0738,  3.8327]])
tensor([[ 1.5105e+00,  4.1851e+00],
        [ 4.7792e+00,  2.5850e+00],
        [ 1.5105e+00,  4.1851e+00],
        [ 3.7694e+00, -4.5000e+00],
        [ 1.5017e+01, -1.0808e+01],
        [ 5.5787e+00,  3.2248e+00],
        [-6.4038e+00,  5.5305e+00],
        [ 8.0021e+00, -1.9996e+00],
        [ 3.1047e+00, -1.7132e-01],
        [ 2.9081e+00,  4.6644e-02],
        [ 4.1169e+00, -3.1344e-01],
        [ 5.5827e+00, -2.3905e+00],
        [ 3.8266e+00,  4.1068e-01],
        [-6.0386e+00,  7.4097e+00],
        [ 1.6558e+01, -1.1451e+01],
        [ 1.6719e+00,  7.6938e-03],
        [ 2.4026e+00,  2.9545e+00],
        [-1.4992e+00,  3.5902e+00],
        [ 3.7246e+00,  4.4174e+00],
        [-2.2101e+00,  2.2723e+00],
    

KeyboardInterrupt: 